### HTML 데이터 파싱을 이용한 크롤링
- 네이버 연관 검색어
    - 1. 웹페이지분석: url찾기
    - 2. 요청 -> 응답 : html(str)
    - 3. html(str) -> BeautifulSoup obj (css-selector) -> DF

In [39]:
import requests

from bs4 import BeautifulSoup

In [40]:
# 1. 웹페이지 분석: URL
query = "삼성전자"
url = "https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query={}".format(
    query)
print(url)

https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=삼성전자


In [41]:
# request -> response : HTML(str)
response = requests.get(url)
response

<Response [200]>

In [42]:
# 3. HTML(str) -> BeautifulSoup obj(css-select) -> DF
dom = BeautifulSoup(response.content, "html.parser")
type(dom)

bs4.BeautifulSoup

In [43]:
# select :  여러개의 element 객체를 리스트로 가져옴
# select_one : 하나의 element 객체를 가져옴

In [44]:
#nx_related_keywords > dl > dd.lst_relate._related_keyword_list > ul
elements = dom.select("._related_keyword_ul > li")
len(elements)

10

In [45]:
keywords = []
for element in elements:
    keywords.append(element.text.strip())
    
print(keywords)

['삼성', '삼성물산', '삼성전자우', '삼성주식', '삼성생명', '카카오', '이건희', '빅히트', '현대자동차', 'sk 하이닉스']


In [46]:
keywords = [element.text.strip()for element in elements]
print(keywords)

['삼성', '삼성물산', '삼성전자우', '삼성주식', '삼성생명', '카카오', '이건희', '빅히트', '현대자동차', 'sk 하이닉스']


In [47]:
# 함수로 만들기
def naver_relational_keywords(query):
    url = "https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query={}".format(query)
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select("._related_keyword_ul > li")
    return [element.text.strip()for element in elements]

In [48]:
naver_relational_keywords("코로나")

['코로나 2.5단계 연장',
 '코로나 라이브',
 '코로나 백신',
 '코로나 확진자',
 '코로나 1단계',
 '코로나 2단계',
 '코로나 부산',
 '부산 코로나',
 '코로나 증상',
 '독감주사 사망']

#### 다음 뉴스 데이터
- 다음 뉴스의 메인 페이지 15개 기사의 제목, 링크, 내용 수집

In [49]:
#1. URL
url = "https://news.daum.net/"

In [50]:
#2. HTML(str)
response = requests.get(url)
response

<Response [200]>

In [51]:
#3. BS
dom = BeautifulSoup(response.content, "html.parser")

In [52]:
elements = dom.select(".list_headline > li")
len(elements)

15

In [53]:
datas = []

for element in elements:
    datas.append({
        "title": element.select_one('.tit_g').text.replace("\n", ""),
        "link": element.select_one('a').get("href")
        })
articles_df =pd.DataFrame(datas)
articles_df.tail(2)

,title,link
13,"""서민증세냐"" 역풍 우려..전국 주택 97% 재산세 감면되나머니투데이",https://news.v.daum.net/v/20201028160500107
14,예술촌으로 변신한 전주 성매매 집결지..112 신고도 줄었다연합뉴스,https://news.v.daum.net/v/20201028160450099


In [54]:
# content 데이터 추가

In [96]:
link = articles_df.loc[1]["link"]
print(link)

https://news.v.daum.net/v/20201028161637672


In [97]:
response = requests.get(link)
dom = BeautifulSoup(response.content, "html.parser")

In [98]:
content = dom.select_one("#harmonyContainer").text.strip().replace("\n","")
content[:100]

'‘3억 대주주’에 뿔난 동학개미與 자본시장특위 "유예해야...현장 혼란 우려"일각선 "부자 증세, 굳이 손볼 필요있나" 반대 당 지도부, 시장 동향 살피며 판단 유예법 개정에는 부'

In [99]:
dom.select_one(".summary_view").text


AttributeError: 'NoneType' object has no attribute 'text'

In [88]:
def get_content(link):
    response = requests.get(link)
    dom = BeautifulSoup(response.content, "html.parser")
    content = dom.select_one(".summary_view").text
    
    return content

In [89]:
articles_df["contents"] = articles_df["link"].apply(get_content)

AttributeError: 'NoneType' object has no attribute 'text'

In [81]:
articles_df.head(5)

,title,link,contents
0,홍콩시위 지지한다더니..美 영사관 뛰어든 4명 망명 거부,https://news.v.daum.net/v/20201028161801723,"[ 홍콩주재 미국 총영사관에 활동가 4명 들어와 , [], 망명요구 했지만 거절 당..."
1,동학개미 업은 민주당 '홍남기' 압박에도..'대주주 3억' 바꾸기 쉽지 않을 듯조선비즈,https://news.v.daum.net/v/20201028161637672,None
2,"'보안 우선' 국회 정보위, 종이 없는 상임위 동참..노트북 설치뉴시스",https://news.v.daum.net/v/20201028161607648,"[ 기밀성 고려 종이 책자 고수..이젠 전자문서 도입, [], 전해철 ""자료·보고서..."
3,"5·18법 찬성하자니..국민의힘 고민 ""개인 판단을 처벌""뉴스1",https://news.v.daum.net/v/20201028161433567,"[ 당 공식 입장 없지만 내부에서는 민주당 법안 내용 문제점 지적 목소리 커, []..."
4,"'지옥철' 지하철 9호선, '8량화' 사실상 무산한국경제",https://news.v.daum.net/v/20201028161402537,None
